In [1]:
using Pkg
Pkg.activate("pidc_env")

  Activating project at `/data/gpfs/projects/punim0638/stephenz/locaTE-paper/examples/mESC/comparisons/pidc/pidc_env`


In [2]:
using NetworkInference
using DataFrames
using Plots
using CSV
using NPZ
using StatsBase
using LinearAlgebra

In [3]:
DATA_PATH="../../../../data/mESC/"
idx = Colon()
X = npzread(string(DATA_PATH, "X.npy"))[:, idx]
try
    global genes = Array(CSV.read(string(DATA_PATH, "genes.txt"), DataFrame)[:, 2])[idx];
catch e
    global genes = ["gene_$i" for i = 1:size(X, 2)];
end
df = DataFrame(X')
insertcols!(df, 1, :gene => genes)
CSV.write("X.csv", df, delim = "\t")

"X.csv"

In [4]:
n = infer_network("X.csv", PIDCNetworkInference(), discretizer = "bayesian_blocks");
w = [e.weight for e in n.edges]
e = [(e.nodes[1].label, e.nodes[2].label) for e in n.edges];
A = zeros(size(X, 2), size(X, 2))
for (w_, e_) in zip(w, e)
    A[findfirst(x -> x == e_[1], genes)[1], 
      findfirst(x -> x == e_[2], genes)[1]] = w_
end
A = A + A';
npzwrite("G_pidc.npy", A)

Getting nodes...
Inferring network...
